# Machine learning algorithms and applications on real world data

In this notebook, we will go through a selection of common supervised ML models used for classification. You will also learn how a typical data scientist might go about to do machine learning on real world datasets. As you'll see, most of the time is actually not spent doing machine learning, but rather in preparing, formatting, cleaning and understanding the data. 

This notebook is in part based on [this](https://nilearn.github.io/auto_examples/02_decoding/plot_haxby_anova_svm.html#sphx-glr-auto-examples-02-decoding-plot-haxby-anova-svm-py) and [this](https://nilearn.github.io/auto_examples/02_decoding/plot_haxby_different_estimators.html#sphx-glr-auto-examples-02-decoding-plot-haxby-different-estimators-py) tutorial from `nilearn`.

As usual we start by importing the required libraries. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

from sklearn.metrics import accuracy_score, roc_curve, auc, plot_roc_curve
from sklearn.metrics import classification_report

# XGBOOST

Above we saw RFs as an example of ensemble learning. The specific technique is known as *bagging* (bootstrap-aggregating). Now we will inspect a model which is based on another related method: *boosting*. Boosting relies on training a model in distinct periods, at each step evaluating which points are more difficult to classify correctly. These difficult points is given extra weight in the next round, incentivising the model to learn those points particularly. 

XGBOOST is currently not built in to `scikit-learn`, but distributed in its own sklearn-compatibable library:

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, model.predict(X_test))

In [ ]:
plot_roc_curve(model, X_test, y_test)

# Real world application of supervised classification

## "Reading thoughts" from brain scans

The most common type of functional magnetic resonance imaging (fMRI) works by recording magnetic field fluctuations inside brains. Blood oxygenation levels is the factor being measured, and is indicative of regional brain activity. Thoughts and other neural processes yields different patterns of brain activity, and thus theory predicts we should be able to discriminate mental states from fMRI scans. This is known as **neural decoding**.

### The Haxby experiment
A pioneering experiment in this field was performed by Haxby et al. (2001), in which they scanned subjects under different visual stimuli: pictures of houses and pictures of human faces (among others). Here we will reproduce the experiment. The library `nilearn` has built in everything we need. 

In [ ]:
from nilearn import datasets
from nilearn.plotting import show

We have to download the dataset.

In [ ]:
haxby_dataset = datasets.fetch_haxby(fetch_stimuli=True) # only the stimulus, 
stimulus_info = haxby_dataset.stimuli # list of paths for images

In [ ]:
# The various categories
stimulus_info.keys()

#### We can look at the images viewed by the test subjects.

In [ ]:
# plot the stimulus 
for stim_type in stimulus_info:
    # skip control images, there are too many
    if stim_type != 'controls':

        file_names = stimulus_info[stim_type]

        fig, axes = plt.subplots(6, 8)
        fig.suptitle(stim_type)

        for img_path, ax in zip(file_names, axes.ravel()):
            ax.imshow(plt.imread(img_path), cmap=plt.cm.gray)

        for ax in axes.ravel():
            ax.axis("off")

show()

In [ ]:
print('Mask nifti image (3D) is located at: %s' % haxby_dataset.mask)
print('Functional nifti image (4D) is located at: %s' %
      haxby_dataset.func[0])

In [ ]:
# First we load a volume. 
from nilearn import image
func_filename = haxby_dataset.func[0] # the first test subject
func_0 = image.load_img(func_filename).slicer[:,:,:,0] # the first volume in the 4D fMRI data

In the machine learning jargon, we consider each voxel (3D pixel) as a *feature*. The vast number of voxels in the image poses a challenge (curse of dimensionality), so we want to do some feature selection before training the classifier. But first: 

#### Exercise 6. how many voxels are in the input volume in total?
Tip: use the tab trick on `func_0` to see available attributes and methods. 

In [ ]:
# %load solutions/ex2_6.py
print(f"The zyx-dimensions are {func_0.shape}")

# number of voxels:
np.prod(func_0.shape)

## Around 160K voxels

We have to retrieve the labels for each volume (i.e. *what* was the subject looking at?). We have limited ourselves to only two states, so you will have to do some basic data manipulation of the labels. You should carefully read the code below to understand how the data is formatted to fit into the model (Scikit-learn accepts only labels on certain formats, as a one-dimensional numpy array or a pandas Series object).

**Tip:** print the intermediate dataframes to understand what each line of code achieves.

In [ ]:
# Load target information as string and give a numerical identifier to each
behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

# Restrict the analysis to faces and places
condition_mask = behavioral['labels'].isin(['face', 'house']) # we will only consider two states
y = conditions[condition_mask]

# Confirm that we now have 2 conditions
print(y.unique())

# Record these as an array of sessions, with fields
# for condition (face or house) and run
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

So far we have defined the labels `y`. The next step is to defined the data `X` as a (**samples x features**) matrix. In `nilearn`, masker-objects will apply a mask and extract the data from the region and put into a numpy array, `X`. 

In [ ]:
from nilearn.input_data import NiftiMasker

mask_filename = haxby_dataset.mask

# For decoding, standardizing is often very important
# note that we are also smoothing the data
masker = NiftiMasker(mask_img=mask_filename, smoothing_fwhm=4,
                     standardize=True, memory="nilearn_cache", memory_level=1)
func_filename = haxby_dataset.func[0] # the first test subject
X = masker.fit_transform(func_filename) # applying the mask
X = X[condition_mask] # selecting only the volumes in condition_mask, i.e. only houses and faces

Now X is on the form that we typically represent data (rows as samples and columns as features). 

#### Exercise 7. a) How many samples do we have in X? b) Compare the shape of X and y.

In [ ]:
# %load solutions/ex2_7.py
# answer: 216. Matrices are always represented in the order: (rows, columns).
# 216 samples, i.e. timepoints
print(X.shape)
print(y.shape)

# now it is in the familiar (nrows x ncols) matrix format for machine learning 

Now that the data has been prepared for machine learning, most of the work is behind us. In the next step we can select any classifier and put it to the task. We start with the linear support vector machine (SVM) (called SVC in `sklearn`), which is both effective and fast to train. Because we have roughly 40K features, we select only the most informative ones (this increases both training speed and accuracy). an F-test can help us do so, again available from `sklearn`.

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test,
# namely Anova. When doing full-brain analysis, it is better to use
# SelectPercentile, keeping 1% of voxels
# (because it is independent of the resolution of the data).
from sklearn.feature_selection import SelectPercentile, f_classif
feature_selection = SelectPercentile(f_classif, percentile=1)

# We have our classifier (SVC), our feature selection (SelectPercentile),and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
from sklearn.pipeline import make_pipeline
anova_svc = make_pipeline(feature_selection, svc)

## How to select optimal hyperparameters: Cross validation

The low **n_samples:n_features** ratio incentivices us to use as many samples as possible for training, while minimizing the test size. We can get an unbiased evaluation of our approach with leave-one-out cross validation. The figure below describes the general process when doing model selection. Here we will deviate slightly by not holding out a final test set.

![title](assets/cross_val.jpg)

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

# Define the cross-validation scheme used for validation.
# Here we use a LeaveOneGroupOut cross-validation on the session group
# which corresponds to a leave-one-session-out
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds (i.e. session)
cv_scores = cross_val_score(anova_svc, X, y, cv=cv, groups=session)

# Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# Print the results
print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / len(y.unique())))

NB! on study design: because the data is time-continuous, it would be biased if you tried to predict a new time point sitting right in between two already known time points. Because the measurements were divided into *sessions*, we remove a whole session for validation, avoiding the problem of autocorrelation.

### Which brain area is responsible?

So MVPA allows for prediction of mental states from patterns in activity. Now we can ask a biologically interesting question: what part of the brain is responsible for this discrimination of faces and houses? How might you go about to answer such a question? It is surprisingly simple to begin to answer. We can look at the weights assigned by the model. It is not straight-forward to interpret the weights of a SVM, but the information held in `svc.coef_` can be used to weight the importance of the different features. We use the whole dataset 

In [ ]:
anova_svc.fit(X, y)
#y_pred = anova_svc.predict(X)

In [ ]:
coef = svc.coef_ # the seperating hyperplane is defined by this vector
# reverse feature selection
coef = feature_selection.inverse_transform(coef) # expand to the full 40K voxels
# reverse masking
weight_img = masker.inverse_transform(coef) # Reshape the array of coefs back to 3D to match the MRI volume

# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(func_filename)

# Create the figure
from nilearn.plotting import plot_stat_map, show
plot_stat_map(weight_img, mean_img, title='SVM weights')

# Saving the results as a Nifti file may also be important
#weight_img.to_filename('haxby_face_vs_house.nii')

show()

### Extra exercise. Select another classifier from `sklearn`, and repeat the analysis.  The classifier needs some measure of feature importance, similar to `.coef_` in the SVC.
Hint: read the documentation on the `sklearn` webpages before deciding on a classifier. Not all models will have something like `coef_`.

In [ ]:
## Example with Naive Bayes